In [2]:
import pandas as pd
import os

df = pd.read_json('/home/weisi/TemporalAssessment/data/BIOASQ/BioASQ.json', lines=True)
df.dtypes

id              object
body            object
exact_answer    object
ideal_answer    object
type            object
snippets        object
year             int64
dtype: object

In [3]:

# calculate snippets length
df['snippets_length'] = df['snippets'].apply(len)

intervals = {
    '2013-2015': (2013, 2015),
    '2016-2018': (2016, 2018),
    '2019-2020': (2019, 2020),
    '2021-2022': (2021, 2022),
}

average_lengths = {}


for interval, (start_year, end_year) in intervals.items():
    interval_data = df[(df['year'] >= start_year) & (df['year'] <= end_year)]
    average_length = interval_data['snippets_length'].mean()
    average_lengths[interval] = average_length

# 打印结果
for interval, avg_length in average_lengths.items():
    print(f"{interval}: average snippets length = {avg_length}")


2013-2015: average snippets length = 2927.531100478469
2016-2018: average snippets length = 2784.477501678979
2019-2020: average snippets length = 1649.7530120481927
2021-2022: average snippets length = 2181.5214285714287


In [ ]:
# simplely split the text
def tokenize(text):
    return text.split()

# save splitted tokens
df['tokens'] = df['snippets'].apply(tokenize)

In [4]:

unique_words_by_interval = {}

for interval, (start_year, end_year) in intervals.items():
    interval_data = df[(df['year'] >= start_year) & (df['year'] <= end_year)]
    unique_words = set(word for tokens in interval_data['tokens'] for word in tokens)
    unique_words_by_interval[interval] = unique_words

    print(f"Unique words count for {interval}: {len(unique_words)}")

for interval_a, words_a in unique_words_by_interval.items():
    for interval_b, words_b in unique_words_by_interval.items():
        if interval_a < interval_b:  # avoid compare same pair twice
            intersection = words_a.intersection(words_b)
            print(f"Overlap between {interval_a} and {interval_b}: {len(intersection)} words")

Unique words count for 2013-2015: 54162
Unique words count for 2016-2018: 54129
Unique words count for 2019-2020: 28701
Unique words count for 2021-2022: 37949
Overlap between 2013-2015 and 2016-2018: 19393 words
Overlap between 2013-2015 and 2019-2020: 12997 words
Overlap between 2013-2015 and 2021-2022: 15256 words
Overlap between 2016-2018 and 2019-2020: 13851 words
Overlap between 2016-2018 and 2021-2022: 15977 words
Overlap between 2019-2020 and 2021-2022: 12104 words
